In [ ]:
# !pip install xmltodict

In [3]:
import xml.etree.ElementTree as ET
from pathlib import Path

def parse_grobid_tei(xml_path):
    NS = {'tei': 'http://www.tei-c.org/ns/1.0'}

    tree = ET.parse(xml_path)
    root = tree.getroot()

    references = []

    for bibl in root.findall('.//tei:listBibl/tei:biblStruct', NS):
        entry = {
            'id': bibl.attrib.get('{http://www.w3.org/XML/1998/namespace}id'),
            'title': None,
            'authors': [],
            'journal': None,
            'volume': None,
            'issue': None,
            'pages': None,
            'year': None,
            'doi': None,
            'arxiv': None,
            'raw': None
        }

        # Title
        title_el = bibl.find('.//tei:analytic/tei:title[@type="main"]', NS)
        if title_el is not None:
            entry['title'] = title_el.text

        # Authors
        for author in bibl.findall('.//tei:analytic/tei:author', NS):
            name = author.find('.//tei:persName', NS)
            if name is not None:
                forename_parts = [fn.text for fn in name.findall('tei:forename', NS) if fn.text]
                surname = name.find('tei:surname', NS)
                full_name = ' '.join(forename_parts + [surname.text if surname is not None else ''])
                entry['authors'].append(full_name.strip())

        # Journal title
        journal_el = bibl.find('.//tei:monogr/tei:title[@level="j"]', NS)
        if journal_el is not None:
            entry['journal'] = journal_el.text

        # Volume, issue, pages
        entry['volume'] = get_text(bibl, './/tei:imprint/tei:biblScope[@unit="volume"]', NS)
        entry['issue'] = get_text(bibl, './/tei:imprint/tei:biblScope[@unit="issue"]', NS)
        page_from = get_text(bibl, './/tei:imprint/tei:biblScope[@unit="page"]', NS, attr="from")
        page_to = get_text(bibl, './/tei:imprint/tei:biblScope[@unit="page"]', NS, attr="to")
        entry['pages'] = f"{page_from}-{page_to}" if page_from and page_to else None

        # Date
        entry['year'] = get_text(bibl, './/tei:imprint/tei:date[@type="published"]', NS, attr="when")

        # Identifiers
        for idno in bibl.findall('.//tei:analytic/tei:idno', NS):
            id_type = idno.attrib.get('type')
            if id_type == 'DOI':
                entry['doi'] = idno.text
            elif id_type == 'arXiv':
                entry['arxiv'] = idno.text

        # Raw reference
        raw_note = bibl.find('.//tei:note[@type="raw_reference"]', NS)
        if raw_note is not None:
            entry['raw'] = raw_note.text

        references.append(entry)

    return references

def get_text(root, path, ns, attr=None):
    el = root.find(path, ns)
    if el is not None:
        return el.attrib.get(attr) if attr else el.text
    return None


In [5]:

# Example usage
if __name__ == "__main__":
    xml_file = "/content/1905.01665v1_SS_Good.pdf.tei.xml"  # Path to your GROBID output file
    # xml_file = "/content/0704.0029v2_SS_Bad.pdf.tei.xml"  # Path to your GROBID output file
    if Path(xml_file).exists():
        refs = parse_grobid_tei(xml_file)
        for ref in refs:
            print(ref)
    else:
        print(f"File not found: {xml_file}")


{'id': 'b0', 'title': 'Use Cases for Authentication and Authorization in Constrained Environments', 'authors': ['L Seitz'], 'journal': None, 'volume': None, 'issue': None, 'pages': None, 'year': '2016-01', 'doi': None, 'arxiv': None, 'raw': 'L. Seitz et al., "Use Cases for Authentication and Authorization in Constrained Environments," RFC 7744, IETF, January 2016.'}
{'id': 'b1', 'title': None, 'authors': [], 'journal': None, 'volume': None, 'issue': None, 'pages': None, 'year': '2019-02-14', 'doi': None, 'arxiv': None, 'raw': '--, "Authentication and Authorization for Constrained Environments (ACE) using the OAuth 2.0 Framework (ACE-OAuth)," IETF Draft, February 14, 2019.'}
{'id': 'b2', 'title': 'The OAuth 2.0 Authorization Framework', 'authors': ['D Hardt'], 'journal': None, 'volume': None, 'issue': None, 'pages': None, 'year': '2012-10', 'doi': '10.17487/rfc6749', 'arxiv': None, 'raw': 'D. Hardt et al., "The OAuth 2.0 Authorization Framework," RFC 6749, Standards Track, IETF, October